In [1]:
import tensorflow as tf
import torch
import scipy.io
import tf_keras_vis
from matplotlib import pyplot as plt
from tf_keras_vis.activation_maximization import ActivationMaximization
from tf_keras_vis.activation_maximization.callbacks import Progress
from tf_keras_vis.activation_maximization.callbacks import GifGenerator2D
from tf_keras_vis.activation_maximization.input_modifiers import Jitter, Rotate2D
from tf_keras_vis.activation_maximization.regularizers import TotalVariation2D, Norm
from tf_keras_vis.utils.model_modifiers import ExtractIntermediateLayer, ReplaceToLinear
from tf_keras_vis.utils.scores import CategoricalScore
from tensorflow import keras

# Check does your computer support using GPU
print("GPU_available={}".format(torch.cuda.is_available()))
torch.device('cuda')
export_matlab = {} # To be exported for matlab analysis

GPU_available=True


In [4]:
# This is for K-fold data
for partic_no in range(0,29):
    
    arch_name = "Arch"+str(partic_no+1)
    trial_no = "trial21"
    mode = 'Kfold_roi' # make sure to change the mode to get the correct trained models
    method = 'maximization' # This code sorts/choses the 10 best EEG samples
    model = keras.models.load_model('keras_trained_models/'+trial_no+'/'+mode+'/'+arch_name+'.h5')
    model.summary()


    layer_name = 'LAYER_13'
    layer = model.get_layer(layer_name)
    print(layer.activation)
    #print(layer.filters)


    # EXTRACT MAXIMUM ACTIVATIONS FOR THE CHOOSEN LAYER
    # This instance constructs new model whose output is replaced to `block5_conv3` layer's output.
    extract_intermediate_layer = ExtractIntermediateLayer(index_or_name=layer_name)

    # This instance modify the model's last activation function to linear one.
    replace2linear = ReplaceToLinear()

    # Because change the target you want to visualize, you MUST create 'Score' instance. 
    # Here, our socre function returns the values corresponding to 63rd, 132nd and 320th filters of the layer.
    model_classes = [0,1] # 0 => Sick Class; 1 => Normal Class
    activations = []

    for class_id in model_classes:

        print("Calculating for Class ", class_id)
        scores = CategoricalScore([class_id])
        activation_maximization = ActivationMaximization(
            model,
            model_modifier=[extract_intermediate_layer, replace2linear], # Please note that `extract_intermediate_layer` has to come before `replace2linear`.
            clone=False
        )

        # Generate maximized activation
        result = activation_maximization(
            score=scores,
            input_modifiers = [Jitter(jitter=4)],
            callbacks=[Progress()],
            steps=100
        )

        activations.append(result)
    field_name = f'field{partic_no+1}'
    export_matlab[field_name] = { 'sick' : activations[0][0][0], 'normal' : activations[1][0][0]} #appending the activations of the 2 classes for each participant\


    # PLOT AND SAVE DIAGRAMS AND DATA FOR MATLAB ANALYSIS
    num_chan = 16
    fig,inner = plt.subplots(num_chan,2, figsize=(20,10))
    for idx, activation in enumerate(activations):
        data = activation[0][0]
        #fig,inner = plt.subplots(num_chan,1)
        title = "Neutral Features" if idx == 1 else "Sickness Features"
        for j in range(num_chan):
            ax = inner[j][idx]

            if j == 1:
                ax.set_title(title)
            
            ax.plot(range(250),data[j,:],'k')
            ax.axis('off')
            ax.set_xticks([])
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
            ax.spines['bottom'].set_visible(False)
            ax.spines['left'].set_visible(False)
            ax.set_yticks([])
            fig.add_subplot(ax)
    plt.show()
    fig.savefig('./Results/Plots/'+trial_no+'/'+mode+'/'+method+'/'+arch_name+'_JITTER_'+layer_name+'.jpg')
    scipy.io.savemat('./Results/Plots/'+trial_no+'/'+mode+'/'+method+'/matrix_jitter_output.mat', {'export_matlab': export_matlab})


OSError: No file or directory found at keras_trained_models/trial21/Kfold_roi/Arch0.h5

In [ ]:
# For single model directory
arch_name = "Arch"
trial_no = "trial22"
model = keras.models.load_model('keras_trained_models/'+trial_no+'/'+arch_name+'.h5')
model.summary()


layer_name = 'LAYER_13'
layer = model.get_layer(layer_name)
print(layer.activation)
#print(layer.filters)


# EXTRACT MAXIMUM ACTIVATIONS FOR THE CHOOSEN LAYER
# This instance constructs new model whose output is replaced to `block5_conv3` layer's output.
extract_intermediate_layer = ExtractIntermediateLayer(index_or_name=layer_name)

# This instance modify the model's last activation function to linear one.
replace2linear = ReplaceToLinear()

# Because change the target you want to visualize, you MUST create 'Score' instance. 
# Here, our socre function returns the values corresponding to 63rd, 132nd and 320th filters of the layer.
model_classes = [0,1] # 0 => Sick Class; 1 => Normal Class
activations = []

for class_id in model_classes:

    print("Calculating for Class ", class_id)
    scores = CategoricalScore([class_id])
    activation_maximization = ActivationMaximization(
        model,
        model_modifier=[extract_intermediate_layer, replace2linear], # Please note that `extract_intermediate_layer` has to come before `replace2linear`.
        clone=False
    )

    # Generate maximized activation
    result = activation_maximization(
        score=scores,
        #input_modifiers = [Jitter(jitter=4)],
        callbacks=[Progress()],
        steps=100
    )

    activations.append(result)
field_name = f'field'
export_matlab[field_name] = { 'sick' : activations[0][0][0], 'normal' : activations[1][0][0]} #appending the activations of the 2 classes for each participant\


# PLOT AND SAVE DIAGRAMS AND DATA FOR MATLAB ANALYSIS
num_chan = 16
fig,inner = plt.subplots(num_chan,2, figsize=(20,10))
for idx, activation in enumerate(activations):
    data = activation[0][0]
    #fig,inner = plt.subplots(num_chan,1)
    title = "Neutral Features" if idx == 1 else "Sickness Features"
    for j in range(num_chan):
        ax = inner[j][idx]

        if j == 1:
            ax.set_title(title)
        
        ax.plot(range(250),data[j,:],'k')
        ax.axis('off')
        ax.set_xticks([])
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.set_yticks([])
        fig.add_subplot(ax)
plt.show()
fig.savefig("./Results/Plots/"+trial_no+'/'+arch_name+"_JITTER_"+layer_name+".jpg")
scipy.io.savemat('./Results/Plots/'+save_to+'/matrix_jitter_output_21_2.mat', {'export_matlab': export_matlab})